<img src="readme.png" alt="Schematic diagram of DigNet generation network" />

## 1.输入数据
### 1.1 
基因表达谱和GRN(数据格式见`Supplement S1`)

- **'net' variable**: Contains the adjacency matrix of network data, a 0-1 weight matrix (Numpy ndarray). Non-0-1 values can also be loaded, sized cell * cell.
- **'exp' variable**: Contains experimental data in DataFrame format, preprocessed scRNA-seq results (CSV format), sized gene * cell.


### 1.2
只输入基因表达谱, 有方法构建一个参考网络(见manuscript, 即readme文档使用的输入格式), 接下来都是使用该格式输入

预处理:参考`CancerDatasets/Create_BRCA_data.py`,基因表达谱是一个csv或xlsx文件,第一行是cell numbers, 第一列是gene symbol IDs



## 2.文件和文件夹

- `pre_train/`: The pre-trained DigNet model provided in this article
- `Cancer_datasets/`: Contains preprocessed results of the E-MTAB-8107 data used in the manuscript
- `pathway/`: sub-function file containing data preprocessing

> 预处理
> MaxMinNormalization(x, Min=0, Max=1): 归一化到(0,1)区间
>
> cal_percent(new_bit_crop,corr_TF_Gene, MI_TF_Gene,net_bit_orig):
1.首先new_bit_origC赋值为net_bit_orig和的new_bit_crop交集,然后计算在new_bit_crop中的比例,记为NUM_ORIG
2.corr_TF_GeneC赋值为corr_TF_GeneC和new_bit_crop的交集,然后计算与net_bit_origC不相交的个数在new_bit_crop中的比例,记为NUM_PCC
3.对MI_TF_Gene的操作与corr_TF_GeneC相同,得到NUM_MI
4.如果NUM_ORIG,NUM_PCC,NUM_MI相加不是100,则缩放使得和为100
5.NUM_PCC+NUM_MIover>50 -> overflow = True
else: overflow = False
return NUM_ORIG, NUM_PCC, NUM_MI, overflow

>load_sergio_count(filename='pathway/simulation/SERGIO_data_node_2000.data',  num=None, logp=True):
用pickle加载filename,如果有num,则选择索引为num的数据,如果logp == True, 则进行log1p变换
return batch

>plot_GRN_percent(network_percent):
画一个"Train_GRN_pecent_bar.pdf"柱形图,
以pathway为横坐标,
NUM_ORIG,NUM_PCC,NUM_MI为纵坐标

>compute_mutual_information(df):
df每行是一个特征
计算df特征间的的mi(互信息)
return mi_matrix

>compare_char(charlist, setlist):
index为setlist中charlist的索引
一般来说index方法只能在list中找一个值的索引,所以charlist可能不是list类型,只是一个值
return index

>calRegnetwork(human_network, GRN_GENE_symbol):
human_network_TF_symbol = human_network第0列
human_network_Gene_symbol= human_network第2列
GRN_GENE_symbol中找调控基因(TF)和靶基因(Gene)
这一段循环方式低效,应该可以改进
return pd.DataFrame(network, columns=['TF', 'Gene'])

>load_KEGG(kegg_file='pathway/kegg/KEGG_all_pathway.pkl'):
如果 pkl 文件存在，则加载它,如果 pkl 文件不存在，则运行 KEGG.py 文件

>high_MI(exp_pca_discretized, exp_pca, net_bit, parm):
计算exp_pca_discretized的互信息,保存在row_MI中,将row_MI的对角线设置为0.然后筛选大于阈值的调控基因和靶基因,把调控基因和靶基因保存到MI_TF_Gene中,并且与net_bit拼接去重.
return net_bit, MI_TF_Gene

>high_pearson(exp_pca, net_bit, parm):
与high_MI中的操作类似,不过未经过离散化.
值得注意的是,mi和pearson相关系数都是对称的,也就是说调控基因和靶基因(有向边)并没有明确的区分?

>from_cancer_create(BRCA_exp_filter_saver, KEGG, parm, lim=200, test_pathway=None, Other_Pathway=None, human_network=None):
1.获得exp表达谱
2.对exp进行标准化
3.net_bit,用calRegnetwork函数在human_network的先验信息中得到,由两列组成:调控基因TF和靶基因Gene
4.net_bit_orig:net_bit的拷贝
5.exp_pca:通过pca降维,列数比exp少一
6.将exp_pca分成256个离散化的区间(0-255),保存为exp_pca_discretized
7.用上面两个函数计算高pearson和MI的调控-靶基因,添加到net_bit
8.创建邻接矩阵adj_matrix
8.1 矩阵形状:基因个数(exp.index) * 基因个数
8.2 有调控关系的(TF-Gene)在矩阵中置1
8.3 经过cal_del_TF_edge(exp.index)函数,将exp分成TF,GENE两个不相交的部分
然后在邻接矩阵adj_matrix中将下标为GENE,TF的点置0(这样就从无向图变成了有向图)
9.predicted_adj_matrix, new_graph = pca_cmi(exp_pca_discretized, net_bit, parm['pmi_percent'], 1):
参考**其他文件**,通过删除cmi较小的边,获得邻接矩阵和图.此处有重要的问题,即离散化的数据用多元正态分布的方法计算cmi不合理.
new_bit_crop是更新后的图的所有边
10.以上有两个邻接矩阵adj_matrix,predicted_adj_matrix.前者是用pearson和mi计算的,后者是用cmi计算的,以下要对两者进行比较
11.分为三种情况,选择边最少的邻接矩阵
11.1 predicted_adj_matrix所有值为0
11.2 (np.sum(adj_matrix) / np.sum(predicted_adj_matrix)) < 0.5:
return exp, adj_matrix, new_row
11.3 else:
return exp, predicted_adj_matrix, new_row
评价:这里用0.5比较,是因为adj_matrix是有向图
predicted_adj_matrix还是一个对称的图,所以如果两个有向图的边相同,predicted_adj_matrix会多一倍的边.但是如果选了predicted_adj_matrix,为什么不把他从对称矩阵(无向)变成有向图?

>matrix2Data(adj_matrix, node_feature, num=0, adj2data=True, log_trans=True, metacell=False, Cnum=100, k=20):
1.1如果node_feature是list
node_feature中选择第num个,转换为tensor,得到x
1.2如果node_feature不是list
直接将node_feature转换为tensor,得到x
log_trans默认True,进行对数变换x = log(1+x)
2.归一化:使用MinMaxScalar,得到x_normalized
3.1如果adj2data是True(默认为True),此时邻接矩阵是0-1编码的,即表示无权图
adj_matrix转换为tensor
indices_tensor是所有的边,每一列代表一条边的头节点和尾节点
num_edges是边的数量
values_tensor是一个全为1的张量,长度是边的数量
3.2如果adj2data是False,此时邻接矩阵是一个稠密的,图中每个点(i,j)都有权重的数组
indeces = 邻接矩阵的所有索引,每一列代表一条边的头节点和尾节点
values = 邻接矩阵的所有值
然后转换成张量,indeces_tensor,values_tensor
4.创建data,一个图,节点特征为x_normalized(形状为(节点数,特征数)),边的索引和权重分别是indeces_tensor,values_tensor,索引为0-(节点数-1)
return data

>load_pathway_mat(file_path, num=0):
从mat类型数据中读取data
node_feature是8个节点的特征,这个8是怎么确定的?
adj_matrix是data中的邻接矩阵
Data1 = matrix2Data(adj_matrix, node_feature, num=num)创建图
return Data1

>create_batch_dataset_from_mat(matnum=100, test=False):
1.如果test = True:测试集
直接加载matnum的mat文件到batch
2.否则:训练集
加载1-matnum的所有mat文件到batch
return batch

>create_batch_dataset_simu(filename='./pathway/simulation/SERGIO_data_node_2000.data', num=None, device=None,test=False,adddata=None, metacell=True, Cnum=100, k=20):


- `pathway/simulation/`: synthetic network and gene expression profile generated by SERGIO
- `discrete/`: model-related configuration files
- `discrete/models/`: Graph Transformer architecture
- `denoising_diffusion_pytorch/`: Contains related sub-function files used for training/testing/initializing DigNet
- `config.py`: Configure hyperparameters for training or testing DigNet
- `DigNet.py`: Contains the process framework of DigNet
- `Download_TF_file.py`: used to download TF list
- `make_final_net.py`: Integrated voting sub-function
- `Tutorial.py`: A quick tutorial for using DigNet

In [5]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA



In [15]:
df = np.arange(10).reshape(2,-1)
df[0,1],df

(1,
 array([[0, 1, 2, 3, 4],
        [5, 6, 7, 8, 9]]))

In [27]:
import networkx as nx
graph = nx.DiGraph()
graph.add_edges_from([['a','b'],['b','c']])

for edge in list(graph.edges()):
    print(edge[0])
    print(set(graph.neighbors(edge[0])))

graph.add_nodes_from([2,2,3,3,4,4,5])
nx.adjacency_matrix(graph).toarray()

a
{'b'}
b
{'c'}


array([[0, 1, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0]], dtype=int32)

In [67]:
import itertools
a = [1,2,3,4]
list(itertools.combinations(a, 3))

[(1, 2, 3), (1, 2, 4), (1, 3, 4), (2, 3, 4)]

## 其他文件
pathway/PCA_CMI.py
>conditional_mutual_info(X, Y, Z=np.array(1)):
计算条件互信息cmi,多元正态分布
如果X,Y是1维,修改形状为(-1,1)的2维数组.此处应该有误,因为cov是将每行当作特征,每列当作样本,(-1,1)的形状算不出协方差,(1,-1)的形状才能算出单个向量的方差.这里没出现报错,输入X,Y应该都是2维数组,列数相同(才能进行cov计算)
Z是常数或向量,用矩阵来说是0维或1维
分以下两种情况:
1.Z是0维,即Z不是一个条件,cmi退化成mi
d1,d2分别是X,Y的协方差的行列式
d3是X和Y拼接后的协方差的行列式
cmi = (1 / 2) * np.log((d1 * d2) / d3)
2.Z是1维,即Z是条件
d1是X,Z的协方差的行列式
d2是Y,Z的协方差的行列式
d3是Z的协方差的行列式
d4是X,Y,Z的协方差的行列式
cmi = (1 / 2) * np.log((d1 * d2) / (d3 * d4))
关于公式推导见**补充知识**
return cmi

>remove_edges(predicted_graph, data, L, theta):
先从predicted_graph中获得边的数量initial_num_edges和所有边edges
遍历每条边edge:
neighbors1 = edge[0]的子节点集合
neighbors2 = edge[1]的子节点集合
neighbors = 以上两个集合取交集,即两个点的公共子节点集合
nhbrs = neighbors的拷贝
T = 公共子节点个数
如果T < L(阈值)或者edge的两个点相同(即edge是自环边),不进行操作
否则:
在data中根据edge的两个端点找到x,y,他们的形状是(-1,1)
K = 在nhbrs中选择L个的组合
1.如果L == 0:
cmiVal = conditionconditional_mutual_info(x.T, y.T)
这个转置(即(1,-1)的输入)也跳过了上面conditionconditional_mutual_info中不合理的判别
如果cmiVal < theta则删除这条边
2.否则:遍历K中的zgroup
2.1 zgroup中没有数据,则跳过
2.2 data中通过zgroup查找z,即影响xy互信息的条件
cmiVal = conditional_mutual_info(x.T, y.T, z.T)
在对zgroup的遍历中获得最大的cmi,如果小于theta则删除这条边
在对每条边进行判断是否删除的操作后,计算图中边的数量,如果比输入时减少了,return predicted_graph, False
否则return predicted_graph, True

>pca_cmi(data, new_net_bit, theta, max_order, show=False):
创建有向图predicted_graph,节点为data的index
new_net_bit中的两列TF,Gene分别是每条边的头节点,尾节点
L = 0
nochange = False
data = data.T
循环,不断增加L,直到nochange == True
这个循环隐含的假设是,随着L增大,条件越多,cmi值会越小,导致不断有边被删除,直到所有边的端点的cmi大于theta,不能被删除.
但是查找资料:条件越多,条件互信息不一定越小或越大,其变化取决于条件变量与目标变量之间的因果或依赖结构.也就是说cmi不是一个单调的结构,这一段也需要考察.
调用时L = 1,即删除操作只进行一次,也不用考察L更大时的单调性质了.
show参数默认是False,如果为True有一段打印信息,调试时可以使用.
返回邻接矩阵和图结构
return predicted_adjMatrix, predicted_graph



## 补充知识
### 互信息(mi)和条件互信息(cmi)
熵,联合熵,条件熵的定义
$H(x) = -\sum_x p(x)log(p(x))\\
H(x,y) = -\sum_{x}\sum_{y} p(x,y)log(p(x,y))\\
H(x|y) = H(x,y) - H(y)$ 
首先,证明互信息mi的熵表达式和概率表达式等价
`熵表达式`
$mi(x,y) = H(x) + H(y) - H(x,y)\\
=H(x)-H(x|y)= H(y) - H(y|x)$
`概率表达式`
$mi(x,y) = \sum_{x}\sum_{y} p(x,y)log(\frac{p(x,y)}{p(x)p(y)})$
证明:
$$\begin{align*}
mi(x,y) &= H(x) + H(y) - H(x,y) \\
&= -\sum_x p(x)log(p(x))-\sum_y p(y)log(p(y))+\sum_{x}\sum_{y} p(x,y)log(p(x,y)) \\
&= -\sum_x\sum_{y} p(x,y)log(p(x))-\sum_{x}\sum_y p(x,y)log(p(y))+\sum_{x}\sum_{y} p(x,y)log(p(x,y)) \\
&=\sum_{x}\sum_{y} p(x,y)log(\frac{p(x,y)}{p(x)p(y)})
\end{align*}$$

条件互信息cmi
$cmi(x,y|z) =  H(x|z) + H(y|z) - H(x,y|z)\\ = \sum_zp(z)mi(x,y|z)$

关于本文中计算cmi的方法,网上的解释:
条件互信息（CMI）与行列式之间存在一定的联系，但直接使用行列式计算CMI并不常见，需要根据具体情况分析。下面从理论和实际应用两个角度进行说明：


### **1. 理论上的联系**
对于**多元高斯分布**，CMI可以通过协方差矩阵的行列式计算。假设随机变量 \( X, Y, Z \) 服从联合高斯分布，则：
\[
\text{CMI}(X;Y|Z) = \frac{1}{2} \ln \left( \frac{\det(\Sigma_{XZ}) \cdot \det(\Sigma_{YZ})}{\det(\Sigma_{Z}) \cdot \det(\Sigma_{XYZ})} \right)
\]
其中：
- \( \Sigma_{XZ} \) 是 \( X \) 和 \( Z \) 的协方差矩阵
- \( \Sigma_{YZ} \) 是 \( Y \) 和 \( Z \) 的协方差矩阵
- \( \Sigma_{Z} \) 是 \( Z \) 的协方差矩阵
- \( \Sigma_{XYZ} \) 是 \( X, Y, Z \) 的联合协方差矩阵

**推导思路**：
高斯分布的微分熵为 \( H(X) = \frac{1}{2} \ln \left( (2\pi e)^n \det(\Sigma_X) \right) \)，代入CMI的熵表达式：
\[
\text{CMI}(X;Y|Z) = H(X|Z) + H(Y|Z) - H(X,Y|Z)
\]
通过行列式的比值化简可得上述公式。


### **2. 离散数据的CMI与行列式**
对于**离散数据**，CMI的定义基于概率分布，而非协方差矩阵，因此**无法直接使用行列式计算**。例如：
\[
\text{CMI}(X;Y|Z) = \sum_{x,y,z} P(x,y,z) \log \frac{P(x,y|z)}{P(x|z) \cdot P(y|z)}
\]
此时需要通过统计频率估计概率分布，而非行列式。

### 总结
本文假设变量都服从多元高斯分布

In [ ]:
#计算互信息的代码,可以看出用两种方法得到的mi差别很大
#注,mutual_info_score仅适用于离散数据,基于行列式的计算方法仅适用于多元高斯分布,
#所以两种方法可比性不高,但是本文中两种方式都出现了,需要考察
from sklearn.metrics import mutual_info_score
x = np.random.randn(20)
y = np.random.randn(20)
#离散化
bins = np.linspace(min(x),max(x),21)
bins1 = np.linspace(min(y),max(y),21)
labels = list(range(20))
x_discrete = pd.cut(x,bins = bins,labels = labels,include_lowest=True)
y_discrete = pd.cut(y,bins = bins1,labels = labels,include_lowest=True)
mi1 = mutual_info_score(x_discrete.to_list(),y_discrete.to_list())
d1 = np.cov(x_discrete)
d2 = np.cov(y_discrete)
d3 = np.linalg.det(np.cov(x_discrete,y_discrete))
mi2 = 1/2 * np.log((d1 * d2)/d3)
mi1, mi2

(1.7388948450374015, 0.04718490304900292)